# Initial Data Exploration
## UCI Online Retail II Dataset

**Objective**: Perform initial exploration of raw transaction data to understand structure, quality, and patterns.

**Tasks**:
1. Load raw data and inspect basic properties
2. Analyze data quality issues (missing values, data types)
3. Explore transaction patterns and distributions
4. Identify potential challenges for modeling
5. Document initial observations

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Load Raw Data

In [ ]:
# Load raw online retail data
df = pd.read_excel('../data/raw/online_retail_II.xlsx', sheet_name='Year 2010-2011')
print(f"Dataset shape: {df.shape}")
print(f"Total transactions: {df.shape[0]:,}")
print(f"Total columns: {df.shape[1]}")

## 2. Data Structure & Info

In [ ]:
# Display first few rows
df.head(10)

In [ ]:
# Data types and non-null counts
df.info()

In [ ]:
# Statistical summary
df.describe()

## 3. Missing Value Analysis

In [ ]:
# Missing value counts
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_pct
})
print(missing_df[missing_df['Missing Count'] > 0].sort_values('Percentage', ascending=False))

In [ ]:
# Visualize missing data
plt.figure(figsize=(10, 6))
sns.heatmap(df.isnull(), cbar=False, yticklabels=False, cmap='viridis')
plt.title('Missing Data Visualization')
plt.tight_layout()
plt.savefig('../eda/01_missing_data_pattern.png', dpi=300, bbox_inches='tight')
plt.show()

## 4. Data Quality Issues

In [ ]:
# Check for negative quantities (returns/cancellations)
negative_qty = df[df['Quantity'] < 0]
print(f"Transactions with negative quantities: {len(negative_qty):,} ({len(negative_qty)/len(df)*100:.2f}%)")

# Check for zero/negative prices
invalid_price = df[df['Price'] <= 0]
print(f"Transactions with zero/negative price: {len(invalid_price):,} ({len(invalid_price)/len(df)*100:.2f}%)")

# Missing customer IDs
missing_customers = df[df['Customer ID'].isnull()]
print(f"Transactions with missing Customer ID: {len(missing_customers):,} ({len(missing_customers)/len(df)*100:.2f}%)")

## 5. Basic Transaction Patterns

In [ ]:
# Transaction distribution over time
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['YearMonth'] = df['InvoiceDate'].dt.to_period('M')

monthly_transactions = df.groupby('YearMonth').size()
plt.figure(figsize=(12, 6))
monthly_transactions.plot(kind='bar')
plt.title('Monthly Transaction Count')
plt.xlabel('Month')
plt.ylabel('Number of Transactions')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('../eda/02_monthly_transactions.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. Customer & Product Analysis

In [ ]:
# Unique counts
print(f"Unique customers: {df['Customer ID'].nunique():,}")
print(f"Unique products: {df['StockCode'].nunique():,}")
print(f"Unique invoices: {df['Invoice'].nunique():,}")
print(f"Unique countries: {df['Country'].nunique():,}")

In [ ]:
# Top 10 countries by transaction count
top_countries = df['Country'].value_counts().head(10)
plt.figure(figsize=(10, 6))
top_countries.plot(kind='barh')
plt.title('Top 10 Countries by Transaction Count')
plt.xlabel('Number of Transactions')
plt.tight_layout()
plt.savefig('../eda/03_top_countries.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Initial Observations & Next Steps

### Key Findings:
1. **Missing Data**: ~20% of transactions lack Customer ID - will need to filter these for customer-level analysis
2. **Returns**: ~1.7% of transactions have negative quantities (cancellations/returns)  
3. **Data Quality**: Some invalid price entries detected
4. **Time Period**: Data spans 12 months (Dec 2009 - Dec 2010)
5. **Geographic**: Primarily UK-based with 38 international markets

### Challenges Identified:
- High percentage of missing Customer IDs limits usable data
- Returns/cancellations need special handling
- Price validation required
- Need to aggregate transaction-level data to customer-level features

### Next Steps:
1. Implement data cleaning pipeline (remove nulls, handle returns, validate prices)
2. Transform to customer-level aggregations
3. Define churn based on temporal behavior
4. Engineer RFM and behavioral features